## 초기 system prompt 설정 및 벡터 store 생성

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from random import randint, choice

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from openai import OpenAI
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

permission = "data_ai"

# 벡터 DB 세팅
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = Chroma(persist_directory=f"./chroma_db/{permission}", embedding_function=embedding_model)

C:\Users\miny0\AppData\Local\Temp\ipykernel_27732\2660580404.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
C:\Users\miny0\AppData\Local\Temp\ipykernel_27732\2660580404.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=f"./chroma_db/{permission}", embed

## 초기 질문 리스트 가져오기

In [2]:
import json

def load_questions_from_jsonl_minimal(path: str, key: str = "question"):
    """JSONL에서 각 라인의 `question` 값만 순서/중복/공백 그대로 추출"""
    questions = []
    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.strip()
            obj = json.loads(line)
            if isinstance(obj, dict) and key in obj:
                questions.append(obj[key])
    return questions

# 사용 예시
initial_questions = load_questions_from_jsonl_minimal(f"./company_questions_{permission}.jsonl")
initial_questions

['모델 성능 평가 보고서에서 Random Forest 모델의 정확도는 어떻게 되나요?',
 '모델 성능 평가 보고서의 4. 성능 지표 및 기준 섹션에서 F1 점수의 기준은 무엇인가요?',
 '모델 성능 평가 보고서의 3. 실험/테스트 절차에서 훈련 데이터와 검증 데이터의 비율은 어떻게 되나요?',
 '모델 성능 평가 보고서에서 제안된 개선 방안 중 데이터 불균형 해소를 위한 방법은 무엇인가요?',
 '모델 성능 평가 보고서의 5. 결과 요약 및 검증 포인트에서 Random Forest 모델의 ROC-AUC 값은 얼마인가요?',
 '모델 성능 평가 보고서의 6. 리스크/한계 및 개선 제안 섹션에서 언급된 과적합의 리스크는 무엇인가요?',
 '모델 성능 평가 보고서의 2. 모델/알고리즘 범위에서 사용된 알고리즘은 어떤 것들이 있나요?',
 '모델 성능 평가 보고서의 4. 성능 지표 및 기준에서 혼동 행렬의 구성 요소는 무엇인가요?',
 '모델 성능 평가 보고서의 1. 프로젝트 개요 및 배경에서 이 보고서의 주요 목표는 무엇인가요?',
 '모델 성능 평가 보고서의 3. 실험/테스트 절차에서 사용된 프로그래밍 언어는 무엇인가요?',
 '모델 성능 평가 보고서의 6. 리스크/한계 및 개선 제안에서 모델 해석성 부족의 한계는 무엇인가요?',
 '모델 성능 평가 보고서의 5. 결과 요약 및 검증 포인트에서 검증 포인트로 제시된 항목은 무엇인가요?',
 '모델 성능 평가 보고서의 4. 성능 지표 및 기준에서 Accuracy의 기준은 얼마인가요?',
 '모델 성능 평가 보고서의 3. 실험/테스트 절차에서 하이퍼파라미터 튜닝이 완료되었는지 확인하는 체크리스트 항목은 무엇인가요?',
 '모델 성능 평가 보고서의 6. 리스크/한계 및 개선 제안에서 데이터 편향의 리스크는 어떤 상황에서 발생할 수 있나요?',
 '데이터 수집 단계에서 데이터 소스 식별 및 목록화는 어떻게 진행되나요?',
 '데이터 전처리에서 결측치 처리 방법은 어떤 기준으로 결정하나요?',
 '데이터 저장 단계에서

In [3]:
def question_chain_setting():
    llm = ChatOpenAI(model="gpt-4.1", temperature=0)

    basic_prompt = PromptTemplate.from_template(
        """
            다음 제공되는 대화 내역을 기반으로 다음에 나올 수 있는 사용자가 궁금해할 만한 후속 질문 하나를 사용자 입장에서 질문하세요.

            대화 내역: {history}

            절대로 AI입장에서의 질문을 만들지 마세요.
            ex) 추가적인 질문이 있으시면 언제든지 말씀해 주시기 바랍니다.
            ex) 추가적으로 궁금한 사항이 있으신가요?
            ex) 데이터 접근 권한 관리 방침에 대해 더 궁금한 사항이 있으면 말씀해 주세요.
            ex) 안녕하세요. 다른 궁금한 점이 있으신가요?
            ex) 안녕하십니까? 도움이 필요하신 부분이 있으신가요?
            ex) ~~ 예시는 다음과 같습니다.


            답변형태는 다음과 같이 해주세요.
            ex) 빌드 결과물의 확인 방법이나 배포 과정에 대해 알고 싶어.
            ex) 문서화 시 유의사항 알려줘.
            ex) 빌드/배포 가이드의 '6. 문서화' 파트에서 문서화해야 할 내용은 뭐야?
            ex) 모델 개발 & 성능 보고서에서 사용된 프로그래밍 언어는 무엇이야?

            <추가 지시사항>
            이전에 나왔던 질문은 최대한 만들지 말고, 연결된 질문으로 조금이라도 다른 질문을 만들어주세요.

        """
    )

    question_chain = basic_prompt | llm | StrOutputParser()

    return question_chain

In [4]:
question_chain = question_chain_setting()

In [5]:
# tool_prompt = f"사용자는 {permission}팀이며 반드시 {permission}_search 툴을 호출하세요."
tool_prompt = f"사용자는 Data AI(데이터 AI)팀에 속한 팀원입니다. 질문이 들어오면 반드시 {permission}_search 툴을 호출하여 문서를 검색하세요."

system_message = f"""
당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 정중하고 사무적인 어조로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 정중하고 사문적인 어조로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 편하게 반말로 물어보더라도, 반드시 정중하고 사무적인 어조로 답변해야 합니다.
8. {tool_prompt}
"""

SYSTEM_PROMPT = {
    "role": "system",
    "content": system_message
}

def generate_one_chat_sample(initial_q: str, max_turns: int = 5):
    messages = [SYSTEM_PROMPT]
    current_q = initial_q
    for turn in range(randint(2, max_turns + 1)):
        # (1) User 질문 추가
        messages.append({"role": "user", "content": current_q})
        print(current_q)

        tool_call = f"<tool_call>{{\"name\": \"{permission}_search\", \"arguments\": {{\"keyword\": \"{current_q}\"}}}}</tool_call>"

        messages.append({"role": "assistant", "content": tool_call})

        # (2) 문서 검색 (vectorstore에서)
        docs = vectorstore.similarity_search(current_q, k=4)
        ref_text = "\n".join([f"{doc.page_content} [[ref{idx+1}]]" for idx, doc in enumerate(docs)])
        ref_text = f"검색 결과:\n-----\n{ref_text}"
        tool_res = f"<tool_response>{ref_text}</tool_response>"
        messages.append({"role": "user", "content": tool_res})

        # (3) assistant 답변 (GPT로 RAG)
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages,
            temperature=0
        )
        assistant_reply = response.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_reply})

        # 다음 질문 생성 여부 랜덤 (5턴이 max면 4턴 미만일때만 질문 생성)
        if turn < max_turns - 1 and choice([True, False]):

            followup = question_chain.invoke({
                "history": messages
            })
            current_q = followup
        else:
            break

    return {"messages": messages}


In [6]:
import time
from tqdm import tqdm

train_dataset = []
for q in tqdm(initial_questions):
    time.sleep(1)  # 1초 대기
    sample = generate_one_chat_sample(q, max_turns=4)
    train_dataset.append(sample)

# 저장
with open(f"qwen3_company_train_dataset_{permission}2.json", "w", encoding="utf-8") as f:
    json.dump(train_dataset, f, ensure_ascii=False, indent=2)


  0%|          | 0/330 [00:00<?, ?it/s]

모델 성능 평가 보고서에서 Random Forest 모델의 정확도는 어떻게 되나요?
Random Forest 모델의 F1 점수와 ROC-AUC 값도 알려줄 수 있어?
Random Forest 모델의 성능 평가에 사용된 데이터셋 분할 비율과 각 데이터셋의 용도에 대해 설명해줄 수 있어?
Random Forest 모델의 하이퍼파라미터는 어떤 값으로 설정되어 있었는지 알려줄 수 있어?


  0%|          | 1/330 [00:13<1:13:34, 13.42s/it]

모델 성능 평가 보고서의 4. 성능 지표 및 기준 섹션에서 F1 점수의 기준은 무엇인가요?


  1%|          | 2/330 [00:16<40:29,  7.41s/it]  

...

 99%|█████████▉| 328/330 [35:34<00:11,  5.74s/it]

모델 학습 결과서의 성능 지표에서 Accuracy는 어떤 의미인가요?
Accuracy 외에 F1 Score나 ROC-AUC 같은 다른 성능 지표들은 각각 어떤 의미가 있는지 설명해줄 수 있어?


100%|█████████▉| 329/330 [35:42<00:06,  6.39s/it]

모델 학습 결과서에서 하드웨어 사양은 어떤 것들이 포함되어 있나요?


100%|██████████| 330/330 [35:45<00:00,  6.50s/it]
